In [1]:
import numpy as np
import uproot, json

# from util.matrix import loadMatrices, baseTransform

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result

def baseTransform(mat, matFromAtoB):
    """
    Reminder: the way this works is that the matrix pointing from pnd to sen0 transforms a matrix IN sen0 back to Pnd
    If you want to transform a matrix from Pnd to sen0, and you have the matrix to sen0, then you need to give
    this function inv(matTo0). I know it's confusing, but that's the way this works.

    Example: matrixInPanda = baseTransform(matrixInSensor, matrixPandaToSensor)
    """
    return matFromAtoB @ mat @ np.linalg.inv(matFromAtoB)

with open("../../input/sensorIDtoSectorID.json", "r") as f:
    sensorIDdict = json.load(f)
    sectorIDlut = np.empty(len(sensorIDdict))

    # create a look up table for fast sensorID -> sectorID translation
    for key, value in sensorIDdict.items():
        sectorIDlut[int(key)] = value

print(f"len: {len(sectorIDlut)}")
print(sectorIDlut.reshape((160, 2)))


len: 320
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [3. 3.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [4. 4.]
 [5. 5.]
 [5. 5.]
 [5. 5.]
 [5. 5.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [7. 7.]
 [7. 7.]
 [7. 7.]
 [7. 7.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [8. 8.]
 [9. 9.]
 [9. 9.]
 [9. 9.]
 [9. 9.]
 [5. 5.]
 [5. 5.]
 [5. 5.]
 [5. 5.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [7. 7.]
 [7. 7.]
 

In [10]:
def readRecoHitsFromRootFiles(filename, maxNoOfFiles=0):

    # make empty 3D (n x 4 x 4) result array.
    # n tracks, 4 reco hits per track (all others are discarded),
    # 4 vales per reco (sector id, x, y, z)
    resultTracks = np.empty((0, 4, 4))

    runIndex = 0
    for arrayDict in uproot.iterate(
        filename,
        [
            "LMDHitsMerged.fSensorID",
            "LMDHitsMerged.fX",
            "LMDHitsMerged.fY",
            "LMDHitsMerged.fZ",
        ],
        library="np",
        allow_missing=True,  # some files may be empty, skip those
    ):
        ids = arrayDict["LMDHitsMerged.fSensorID"]
        recoX = arrayDict["LMDHitsMerged.fX"]
        recoY = arrayDict["LMDHitsMerged.fY"]
        recoZ = arrayDict["LMDHitsMerged.fZ"]
        
        print(f'len before mask: {len(ids)}')
        # print(f'looks like this:\n{arrayDict}')

        # create mask for events that have exactly 4 hits
        mask = [event.size > 3 for event in ids]
        print(f'len after mask: {len(ids[mask])}')

        testarray = ids

        print(f'hmmm...\n{ids}')
        print(f'hmmm...\n{recoX}')
        print(f'hmmm...\n{recoY}')
        print(f'hmmm...\n{recoZ}')

        # make a real 2D array from array[array[ ]]
        maskedIDs = np.stack(ids[mask])

        # calculate module number from sensor IDs
        # use look up table for that
        # thank you TheodrosZelleke for this insanely smart idea
        # https://stackoverflow.com/a/14448935
        sectorIDfromLut = sectorIDlut[maskedIDs]

        # make arrays of arrays to 2d arrays
        xFlat = np.stack(recoX[mask])
        yFlat = np.stack(recoY[mask])
        zFlat = np.stack(recoZ[mask])

        # transpose to assemble and transpose again
        theseTracks = np.array([sectorIDfromLut.T, xFlat.T, yFlat.T, zFlat.T]).T

        # sort every 4-hit combo by z value. do this for every event.
        # this is REQUIRED for the aligner
        # I know it looks weird seeing a list comprehension here instead of something
        # more NumPythonic, but this seems to be fastest after all
        # the numpy version would probably look like this:
        # sortedResultArray = theseTracks[:,theseTracks[:,:,3].argsort()]
        # or 
        # sortedResultArray = np.einsum('iijk->ijk', theseTracks[:,theseTracks[:,:,0].argsort()])
        # and be 20x slower (and require MUCH more memory)
        sortedResultArray = np.array([subarray[subarray[:,3].argsort()] for subarray in theseTracks])

        print(f'so this is what i got now:\n{sortedResultArray}')

        # stack this files' content with the others
        resultTracks = np.vstack((resultTracks, sortedResultArray))

        runIndex += 1
        if runIndex == maxNoOfFiles:
            break

    return resultTracks

# path = "/mnt/himsterData/roklasen/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/geo_misalignmentmisMat-modules/100000/1-100_uncut/no_alignment_correction/"
path = "/media/CacheDrive2TB/Arbeit/PandaRoot/macro/detectors/lmd/testFullChain/mom-15/"
# path = "/media/CacheDrive2TB/Arbeit/PandaRoot/macro/detectors/lmd/testFullChain/mom-1_5/"
rootFileWildcard = "Lumi_recoMerged_*.root:pndsim"

resultArray = readRecoHitsFromRootFiles(path+rootFileWildcard, 1)

alignmentMatices = {}

# translate the reco hits to format for module Aligner
for i in range(10):
    # apply mask so only one sector is chosen
    # careful, this only checks if the first hit
    # is in the correct sector!
    sectorMask = resultArray[:, 0, 0] == i

    # create new empty array thay has the required structure
    sectorTracksXYZ = resultArray[sectorMask][:, :, 1:4]
    nTrks = len(sectorTracksXYZ)

    # assign recos from input array to new array for this sector
    trackVectorForAligner = np.ones((nTrks, 6, 4))
    trackVectorForAligner[:, 2:6, :3] = sectorTracksXYZ[:, 0:4]
    break

    alignmentMatices |= alignSectorICP(trackVectorForAligner, i)

    break


len before mask: 100000
len after mask: 99992
hmmm...
[array([ 38, 194,  22, 160,  31,  78, 234, 200,  71, 118, 274, 240, 314,
        280], dtype=int32)
 array([  0, 169,   1,  28,  35,  40, 210,  40,  40, 236,  68,  41, 232,
        236,  48, 233,  42,  49,  49,  49, 226, 209,  42,  47,  47,  43,
         45,  80,  80,  80, 250, 272, 272, 272, 276, 276, 108,  85, 270,
         88, 273,  91,  90, 248, 251,  82,  87, 120, 308, 308, 120, 290,
        121, 312, 316, 148, 120, 121, 131, 126, 291, 294, 155, 158, 131,
         73,  79, 243], dtype=int32)
 array([172,  37, 194, 199,  13, 161, 164, 160, 212, 224,  77, 224, 228,
        234, 220,  53, 220, 201, 220, 221, 233,  47,  49, 218, 252, 117,
        269, 252, 274, 260, 256,  93, 241, 267, 259, 259,  97, 292, 157,
        309, 314, 300, 296, 133, 291, 294, 281, 302], dtype=int32)
 ...
 array([  2, 171,   9,  11, 176, 176,  24, 162, 198, 184,  30, 194, 193,
        199,   4, 178, 178,  42, 211,  49,  51, 216, 216, 238,  64, 235,
       

ValueError: all input arrays must have the same shape